In [6]:
import pandas as pd
file_path = '../data/Online_Retail.csv'
df = pd.read_csv(file_path, encoding='unicode_escape')

print("--- First 5 rows of the DataFrame ---")
print(df.head())

print("\n--- Data Types and Nulls per Column (df.info()) ---")
df.info()

--- First 5 rows of the DataFrame ---
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/10 8:26       2.55     17850.0  United Kingdom  
1  12/1/10 8:26       3.39     17850.0  United Kingdom  
2  12/1/10 8:26       2.75     17850.0  United Kingdom  
3  12/1/10 8:26       3.39     17850.0  United Kingdom  
4  12/1/10 8:26       3.39     17850.0  United Kingdom  

--- Data Types and Nulls per Column (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Nu

In [16]:
print("--- Null Vote Count and Percentages ---")
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

missing_data = pd.DataFrame({
    'Count' : null_counts,
    'Percentage': null_percentages.round(2)
})

print(missing_data.sort_values(by='Percentage', ascending=False))

--- Null Vote Count and Percentages ---
              Count  Percentage
CustomerID   135080       24.93
Description    1454        0.27
InvoiceNo         0        0.00
StockCode         0        0.00
Quantity          0        0.00
InvoiceDate       0        0.00
UnitPrice         0        0.00
Country           0        0.00


1. Data Governance and Initial Cleaning Decisions
This section addresses key data quality issues identified during the initial diagnostic, specifically focusing on preparing the data for the **RFM Segmentation** which aligns with the business objetive of client retention.

A. Handling Missing Customer IDs (High-Impact Decision)
**Issue:** Approximately **25% of sales records lack a `CustomerID`**.
**Business Impact:** These records cannot be used for critical **retention analysis**, **RFM segmentation**, or calculating **Customer Lifetime Value (CLTV)**, as the buying entity is anonymous.
**Decision Rationale:**
    **For RFM Analysis:** We will **drop** all rows where `CustomerID` is null. This decision ensures the integrity and focus of our client retention strategy
        **For Total Sales/Revenue Analysis:** The original DataFrame (`df`) is retained for reference, ensuring we can still report on total company revenue when needed.

B. Data Type and Integrity Checks
**Issue 1: Dates:** The `InvoiceDate` column is loaded as an `Object` (string) and must be converted to `datetime` to allow for Recency calculations.
**Issue 2: Negative Quantities:** The `Quantity` column contains negative values, which represent **returns or cancellations**. These must be removed from the primary analysis to accurately calculate gross sales and RFM metrics.
**Transformation:** A new column, **`TotalPrice`**, will be calculated as `Quantity * UnitPrice` to represent the total revenue per transaction line, a crucial metric for the 'Monetary' component of RFM.

We will proceed by creating a **clean DataFrame (`df_rfm`)** ready for advanced analytics.

In [17]:
df_rfm = df.dropna(subset=['CustomerID'])

df_rfm['CustomerID'] = df_rfm['CustomerID'].astype(int)

df_rfm['InvoiceDate'] = pd.to_datetime(df_rfm['InvoiceDate'])

df_rfm = df_rfm[df_rfm['Quantity'] > 0]
df_rfm = df_rfm[df_rfm['UnitPrice'] > 0]


df_rfm['TotalPrice'] = df_rfm['Quantity'] * df_rfm['UnitPrice']

print(f"Original Data Size: {len(df)}")
print(f"Clean RFM Data Size: {len(df_rfm)}")
print("\nFinal Data Types Check for RFM Analysis:")
df_rfm.info()

C:\Users\fero\AppData\Local\Temp\ipykernel_18552\34731733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rfm['CustomerID'] = df_rfm['CustomerID'].astype(int)
C:\Users\fero\AppData\Local\Temp\ipykernel_18552\34731733.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_rfm['InvoiceDate'] = pd.to_datetime(df_rfm['InvoiceDate'])


Original Data Size: 541909
Clean RFM Data Size: 397884

Final Data Types Check for RFM Analysis:
<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  int64         
 7   Country      397884 non-null  object        
 8   TotalPrice   397884 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 30.4+ MB


C:\Users\fero\AppData\Local\Temp\ipykernel_18552\34731733.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rfm['InvoiceDate'] = pd.to_datetime(df_rfm['InvoiceDate'])
